In [1]:
import re
from multiprocessing import Pool


In [2]:
input = '''Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3
'''

# with open('input') as f:
#     input = f.read()


In [3]:
row = 10
r = 20
# row = 2000000
# r = 4000000


In [4]:
sensors = list()
beacons = list()
radii = list()

x_min = x_max = 0

searchSpace = set()


In [5]:
for line in input.splitlines():
    coordinates = re.findall('x=(-?\d+), y=(-?\d+)', line)
    coordinates = [(int(x), int(y)) for x, y in coordinates]

    pos_sensor = coordinates[0]
    pos_beacon = coordinates[1]
    radius = sum(tuple(map(lambda a, b: abs(a - b), pos_sensor, pos_beacon)))

    sensors.append(pos_sensor)
    radii.append(radius)
    beacons.append(pos_beacon)

    radiusOnY = radius - abs(pos_sensor[1] - row)
    currMin = pos_sensor[0] - radiusOnY
    currMax = pos_sensor[0] + radiusOnY
    x_min = min(x_min, currMin)
    x_max = max(x_max, currMax)
    
    # create the search space
    # TODO: use a more space-efficient way to represent the sensor's search area
    x = 0
    y = radius + 1
    while y > -1:
        # [Q1, Q2, Q3, Q4]
        newPoints = [(pos_sensor[0] + x, pos_sensor[1] + y), (pos_sensor[0] - x, pos_sensor[1] + y), (pos_sensor[0] - x, pos_sensor[1] - y), (pos_sensor[0] + x, pos_sensor[1] - y)]
        newPoints = [p for p in newPoints if 0 <= p[0] <= r and 0 <= p[1] <= r]
        searchSpace.update(newPoints)

        x += 1
        y -= 1


In [6]:
def is_free_position(pos):
    # position already contains a beacon ('B') or a sensor ('S')
    if pos in beacons or pos in sensors:
        return False
    
    # check the range of all sensors ('#')
    # TODO: brute-force might be too slow on full input
    for pos_sensor, radius in zip(sensors, radii):
        distance = sum((abs(pos_sensor[0] - pos[0]), abs(pos_sensor[1] - pos[1])))
        if distance <= radius:
            return False
    
    # out of any sensor range, beacon positioning is possible ('.')
    return True


In [7]:
# freePositions = list(filter(is_free_position, searchSpace))
p = Pool()
freePositions = [
    position for position, isFree
    in zip(searchSpace, p.imap(is_free_position, searchSpace))
    if isFree
]

assert len(freePositions) == 1
distressBeacon = freePositions[0]
tuningFrequency = distressBeacon[0] * 4000000 + distressBeacon[1]


In [8]:
print(tuningFrequency)


56000011
